In [2]:
import pathlib
import spacy
import numpy as np
import re
import os
from spacy.lang.zh.stop_words import STOP_WORDS
import jieba
import zh_core_web_lg
import sklearn
from sklearn import neighbors
from scipy import spatial
from sklearn.metrics import classification_report
import string


c:\users\xiaok\appdata\local\programs\python\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pathlib 

docs = []
labels = []


In [ ]:
#preset labels 
label_names = ['广告', '色情', '辱骂', '争吵', '水贴', '交易', '挖坟', '政治', '隐私', '虚假', '歧视', '求助', '引战', '授权']

file = pathlib.Path(r"C:\Users\xiaok\Desktop\598sc\mingxing.txt").read_text(encoding='utf-8')
docs = file.split('。')

In [62]:
#Clean text and remove some common words appeal in Chinese 
punctuation_string = string.punctuation
punc = '＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､\u3000、〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〾〿–—‘’‛“”„‟…‧﹏﹑﹔·！？｡。'
def clean_text(text):
    text= text.strip()
    text = " ".join(re.findall(r'\b\w+\b', text))
    text = re.sub('[{}]'.format(punctuation_string),"",text)
    text = re.sub('[\da-zA-Z、一二三四五六七八九十条款]','',text) # [0-9]
    return text

In [63]:
doc = clean_text(doc)

In [64]:
#word embedding
def embed(tokens, nlp):
    """Return the centroid of the embeddings for the given tokens.

    Out-of-vocabulary tokens are cast aside. Stop words are also
    discarded. An array of 0s is returned if none of the tokens
    are valid.

    """
    lexemes = (nlp.vocab[token] for token in tokens)

    vectors = np.asarray([
        lexeme.vector
        for lexeme in lexemes
        if lexeme.has_vector
        and not lexeme.is_stop
        and len(lexeme.text) > 1
    ])
    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
        
    else:
        width = nlp.meta['vectors']['width']  # typically 300
        centroid = np.zeros(width)

    return centroid

In [93]:
#find closet label to the current doc 
nlp = spacy.load("zh_core_web_lg")

neigh = neighbors.NearestNeighbors(n_neighbors=1, metric=spatial.distance.cosine)
neigh.fit(label_vectors)

label_vectors = np.asarray([
    embed(label.split(' '), nlp)
    for label in label_names
])

def predict(doc, nlp, neigh):
    tokens = ""
    doc = clean_text(doc)
    tokens = jieba.lcut(doc)
    tokens = ' '.join(tokens)
    tokens = tokens.split(" ")
    while("" in tokens):
        tokens.remove("")
    centroid = embed(tokens, nlp)
    closest_label = neigh.kneighbors([centroid], return_distance=False)[0][0]
    return closest_label

preds = [label_names[predict(doc, nlp, neigh)] for doc in docs]
y_true = label_names
report = classification_report(y_true = preds, y_pred=preds,labels=label_names)
    

c:\users\xiaok\appdata\local\programs\python\python37\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\users\xiaok\appdata\local\programs\python\python37\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\users\xiaok\appdata\local\programs\python\python37\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\users\xiaok\appdata\local\programs\python\python37\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\users\xiaok\appdata\local\programs\python\python37\lib\site-packages\scipy\spatial\distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [95]:
print("Label   \tSupports\tpercentage")
translate = ['Ad', 'Porngraphy', 'Abuse', 'Toxicity', 'Meaningless', 'Trade', 'necro-bump', 'Politics', 'Privacy', 'Falsehood', 'Discrimination', 'Help Request', 'Trolling', 'Authorization']
j = 0
for i in label_names:
    print(translate[j],"   \t" ,preds.count(i),"\t\t","{:.01%}".format(preds.count(i)/len(preds)))
    j = j + 1

Label   	Supports	percentage
Ad    	 60 		 18.9%
Porngraphy    	 14 		 4.4%
Abuse    	 29 		 9.1%
Toxicity    	 19 		 6.0%
Meaningless    	 10 		 3.2%
Trade    	 10 		 3.2%
necro-bump    	 14 		 4.4%
Politics    	 15 		 4.7%
Privacy    	 45 		 14.2%
Falsehood    	 28 		 8.8%
Discrimination    	 18 		 5.7%
Help Request    	 16 		 5.0%
Trolling    	 3 		 0.9%
Authorization    	 36 		 11.4%
